In [1]:
import os
os.environ['PKG_CONFIG_PATH'] = '/ocean/projects/asc170022p/mtragoza/mambaforge/envs/4DCT/lib/pkgconfig'

In [2]:
import numpy as np
import fenics as fe
from fenics import grad, dot, dx, ds
import matplotlib.pyplot as plt

--------------------------------------------------------------------------
but there are no active ports detected (or Open MPI was unable to use
them).  This is most certainly not what you wanted.  Check your
cables, subnet manager configuration, etc.  The openib BTL will be
ignored for this job.

  Local host: dv001
--------------------------------------------------------------------------


In [3]:
n_nodes = 81
n_elements = n_nodes - 1
mesh = fe.UnitSquareMesh(n_elements, n_elements)
V = fe.FunctionSpace(mesh, 'Lagrange', 1)
mesh_spacing = 1 / n_elements
mesh_spacing

0.0125

In [4]:
# initial and Dirichlet boundary condition
ic_expression = fe.Constant(0)
u0 = fe.interpolate(ic_expression, V)
u1 = fe.interpolate(ic_expression, V)

def boundary(x, on_boundary):
    return on_boundary and fe.near(x[1], 0, 1e-14)

omega = 50 # Hz
bc_expression = fe.Expression(f'sin(2 * 3.1415 * omega * t)', degree=2, t=0, omega=omega)
bc = fe.DirichletBC(V, bc_expression, boundary)

In [5]:
# source term and Neumann boundary condition
f = fe.Expression('0', degree=0)
g = fe.Expression('0', degree=1)

In [6]:
# wave speed and time stepping parameters

# specify wave speed in meters per second (from physical constants)
rho = 1000 # kg / m^3
mu  = 4000 # Pa = N / m^2 = (kg m/s^2) / m^2 = kg / (m s^2)
c = np.sqrt(mu / rho) # (kg / (m s^2)) / (kg / m^3) = m^3 / (m s^2) = (m / s)^2

#c = 2 # meters / second

# dimensionalize the wave speed
L = 0.2 # side length in meters
c /= L # side lengths per second
print(c)

dt = 1e-4 # seconds

# check CFL condition
cfl_number = c * dt / mesh_spacing
print(cfl_number)
assert cfl_number <= 1, f'CFL condition violated ({c} x {dt} / {mesh_spacing} <= 1)'

total_time = 0.4 # seconds
n_steps = np.ceil(total_time / dt).astype(int)
n_steps

10.0
0.08


4000

In [7]:
# variational form
u = fe.TrialFunction(V)
v = fe.TestFunction(V)
a = (
    u * v * dx + 
    (c * dt)**2 * dot(grad(u), grad(v)) * dx
)
L = (
    (2 * u1 - u0 + dt**2 * f) * v * dx + 
    (g * v) * ds
)

In [8]:
# damped version

class AbsorbingBoundary(fe.UserExpression):
    def value_shape(self):
        return ()
    def eval(self, values, x):
        if x[0] < 0.05 or x[0] > 0.95 or x[1] > 0.95:
            values[0] = -1000
        else:
            values[0] = -10

k = fe.interpolate(AbsorbingBoundary(), V)

u = fe.TrialFunction(V)
v = fe.TestFunction(V)
a = (
    (1 - k * dt) * u * v * dx + 
    (c * dt)**2 * dot(grad(u), grad(v)) * dx
)
L = (
    ((2 - k * dt) * u1 - u0 + dt**2 * f) * v * dx + 
    (g * v) * ds
)

In [9]:
# get mapping from dofs to array indices
xyz = V.tabulate_dof_coordinates()
x = xyz[:,0]
y = xyz[:,1]
print(xyz.shape)

inds = np.argsort([n_nodes * x + y for x,y in xyz])
xyz[inds]

(6561, 2)


array([[0.    , 0.    ],
       [0.    , 0.0125],
       [0.    , 0.025 ],
       ...,
       [1.    , 0.975 ],
       [1.    , 0.9875],
       [1.    , 1.    ]])

In [10]:
u0.interpolate(ic_expression)
u1.interpolate(ic_expression)

u = fe.Function(V)
array = np.zeros((n_steps + 1, n_nodes, n_nodes))

t = 0
for i in range(n_steps):
    t += dt
    bc_expression.t = t
    fe.solve(a == L, u, [bc])
    u0.assign(u1)
    u1.assign(u)

    values = u.vector().get_local()
    array[i+1,...] = values[inds].reshape((n_nodes, n_nodes))

    if i % 10 == 0:
        print(f'Iteration {i+1} / {n_steps}')

Solving linear variational problem.
Iteration 1 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 11 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 21 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solvi

Solving linear variational problem.
Iteration 221 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 231 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 241 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
S

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 441 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 451 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 461 / 4000
Solving linear variational problem.
Solving linear variational problem.
S

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 661 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 671 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 681 / 4000
Solving linear variational problem.
Solving linear variational problem.
S

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 881 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 891 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variati

...

Solving linear variational problem.
Iteration 3051 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3061 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3071 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3271 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3281 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3291 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3491 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3501 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3511 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem

Solving linear variational problem.
Solving linear variational problem.
Iteration 3711 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3721 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3731 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3931 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3941 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3951 / 400

In [11]:
import xarray as xr
import hvplot.xarray

array = xr.DataArray(
    array,
    dims=['t', 'x', 'y'],
    coords={
        't': np.arange(0, n_steps + 1) * dt,
        'x': np.linspace(0, 1, n_nodes),
        'y': np.linspace(0, 1, n_nodes)
    }
)
array.hvplot(groupby=['t'], x='x', y='y', data_aspect=1, clim=(-2, 2), cmap='seismic', widget_type='scrubber')

Row
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_type='scrubber')
    [1] WidgetBox(align=('end', 'start'))
        [0] Player(end=4000, width=550)

In [17]:
array.hvplot(groupby=['t', 'x'], x='y', ylim=[-2, 2], widget_location='bottom')

Column
    [0] HoloViews(DynamicMap, height=300, sizing_mode='fixed', widget_location='bottom', width=700)
    [1] WidgetBox(align=('center', 'end'))
        [0] DiscreteSlider(margin=(20, 20, 5, 20), name='t', options=OrderedDict([('0', ...]), value=0.0, width=250)
        [1] DiscreteSlider(margin=(5, 20, 20, 20), name='x', options=OrderedDict([('0', ...]), value=0.0, width=250)

In [12]:
array

<xarray.DataArray (t: 4001, x: 81, y: 81)>
array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 3.14098330e-02, -2.54648891e-03, -1.37541374e-05, ...,
          1.35237056e-64,  7.61137918e-66, -1.68067778e-65],
        [ 3.14098330e-02, -8.68574545e-03,  1.47234084e-03, ...,
          4.50396565e-64, -1.18284360e-64,  3.10481520e-65],
        [ 3.14098330e-02, -7.82132677e-03,  2.18788495e-03, ...,
         -1.60636481e-63,  1.34570144e-64,  2.74169066e-65],
...
        [-3.51139238e-02, -4.16045968e-01, -4.82384654e-01, ...,
          1.10084026e-03, -3.11341056e-03, -4.73141973e-03],
        [-3.51139238e-02, -4.09031497e-01, -4.51429588e-01, ...,
         -1.84187369e-03, -4.54469318e-03, -5.57407622e-03],
        [-3.51139238e-02, -4.12622288e-01, -4.47321324e-01, ...,
         -2.90713143e-03, -5.08087127e-03, -5.97021164e-03]],

       [[-3.70613512e-03, -3.64843636e-01, -4.06948164e-01, ...,
         -6.71760229e-04, -2.11817112e-03, -2.51918882e-03],
        [-3.70613512e-03, -3.81138524e-01, -4.31562568e-01, ...,
         -1.11161467e-04, -1.88738308e-03, -2.45961289e-03],
        [-3.70613512e-03, -3.99956507e-01, -4.75295032e-01, ...,
          1.67913892e-03, -1.05872194e-03, -1.97139923e-03],
        ...,
        [-3.70613512e-03, -3.98210969e-01, -4.76646863e-01, ...,
          1.30864215e-03, -2.94138697e-03, -4.57607352e-03],
        [-3.70613512e-03, -3.92181760e-01, -4.47163335e-01, ...,
         -1.68298267e-03, -4.45368283e-03, -5.51143287e-03],
        [-3.70613512e-03, -3.95868532e-01, -4.43405974e-01, ...,
         -2.76604567e-03, -5.01957229e-03, -5.94898137e-03]]])
Coordinates:
  * t        (t) float64 0.0 0.0001 0.0002 0.0003 ... 0.3997 0.3998 0.3999 0.4
  * x        (x) float64 0.0 0.0125 0.025 0.0375 ... 0.9625 0.975 0.9875 1.0
  * y        (y) float64 0.0 0.0125 0.025 0.0375 ... 0.9625 0.975 0.9875 1.0

In [ ]:
%%time
import os
import subprocess

def create_video(video_name, array):
    
    print('Creating video frames...')
    os.makedirs(video_name, exist_ok=True)
    for i, frame in enumerate(array):
        frame_file = video_name + f'/frame{i:04d}.png'
        plt.imshow(frame.T, vmin=-2, vmax=2, cmap='seismic')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.savefig(frame_file)
        print(frame_file)
        
    print('Creating video...')
    os.chdir(video_name)
    subprocess.call([
        'ffmpeg', '-framerate', '30', '-i', 'file%04d.png', '-r', '30', '-pix_fmt', 'yuv420p', f'{video_name}.mp4'
    ])

create_video('my_video', array.values)

Creating video frames...
my_video/frame0000.png
my_video/frame0001.png
my_video/frame0002.png
my_video/frame0003.png
my_video/frame0004.png
my_video/frame0005.png
my_video/frame0006.png
my_video/frame0007.png
my_video/frame0008.png
my_video/frame0009.png
my_video/frame0010.png
my_video/frame0011.png
my_video/frame0012.png
my_video/frame0013.png
my_video/frame0014.png
my_video/frame0015.png
my_video/frame0016.png
my_video/frame0017.png
my_video/frame0018.png
my_video/frame0019.png
my_video/frame0020.png
my_video/frame0021.png
my_video/frame0022.png
my_video/frame0023.png
my_video/frame0024.png
my_video/frame0025.png
my_video/frame0026.png
my_video/frame0027.png
my_video/frame0028.png
my_video/frame0029.png
my_video/frame0030.png
my_video/frame0031.png
my_video/frame0032.png
my_video/frame0033.png
my_video/frame0034.png
my_video/frame0035.png
my_video/frame0036.png
my_video/frame0037.png
my_video/frame0038.png
my_video/frame0039.png
my_video/frame0040.png
my_video/frame0041.png
my_video/

my_video/frame0356.png
my_video/frame0357.png
my_video/frame0358.png
my_video/frame0359.png
my_video/frame0360.png
my_video/frame0361.png
my_video/frame0362.png
my_video/frame0363.png
my_video/frame0364.png
my_video/frame0365.png
my_video/frame0366.png
my_video/frame0367.png
my_video/frame0368.png
my_video/frame0369.png
my_video/frame0370.png
my_video/frame0371.png
my_video/frame0372.png
my_video/frame0373.png
my_video/frame0374.png
my_video/frame0375.png
my_video/frame0376.png
my_video/frame0377.png
my_video/frame0378.png
my_video/frame0379.png
my_video/frame0380.png
my_video/frame0381.png
my_video/frame0382.png
my_video/frame0383.png
my_video/frame0384.png
my_video/frame0385.png
my_video/frame0386.png
my_video/frame0387.png
my_video/frame0388.png
my_video/frame0389.png
my_video/frame0390.png
my_video/frame0391.png
my_video/frame0392.png
my_video/frame0393.png
my_video/frame0394.png
my_video/frame0395.png
my_video/frame0396.png
my_video/frame0397.png
my_video/frame0398.png
my_video/fr

my_video/frame0713.png
my_video/frame0714.png
my_video/frame0715.png
my_video/frame0716.png
my_video/frame0717.png
my_video/frame0718.png
my_video/frame0719.png
my_video/frame0720.png
my_video/frame0721.png
my_video/frame0722.png
my_video/frame0723.png
my_video/frame0724.png
my_video/frame0725.png
my_video/frame0726.png
my_video/frame0727.png
my_video/frame0728.png
my_video/frame0729.png
my_video/frame0730.png
my_video/frame0731.png
my_video/frame0732.png
my_video/frame0733.png
my_video/frame0734.png
my_video/frame0735.png
my_video/frame0736.png
my_video/frame0737.png
my_video/frame0738.png
my_video/frame0739.png
my_video/frame0740.png
my_video/frame0741.png
my_video/frame0742.png
my_video/frame0743.png
my_video/frame0744.png
my_video/frame0745.png
my_video/frame0746.png
my_video/frame0747.png
my_video/frame0748.png
my_video/frame0749.png
my_video/frame0750.png
my_video/frame0751.png
my_video/frame0752.png
my_video/frame0753.png
my_video/frame0754.png
my_video/frame0755.png
my_video/fr

my_video/frame1070.png
my_video/frame1071.png
my_video/frame1072.png
my_video/frame1073.png
my_video/frame1074.png
my_video/frame1075.png
my_video/frame1076.png
my_video/frame1077.png
my_video/frame1078.png
my_video/frame1079.png
my_video/frame1080.png
my_video/frame1081.png
my_video/frame1082.png
my_video/frame1083.png
my_video/frame1084.png
my_video/frame1085.png
my_video/frame1086.png
my_video/frame1087.png
my_video/frame1088.png
my_video/frame1089.png
my_video/frame1090.png
my_video/frame1091.png
my_video/frame1092.png
my_video/frame1093.png
my_video/frame1094.png
my_video/frame1095.png
my_video/frame1096.png
my_video/frame1097.png
my_video/frame1098.png
my_video/frame1099.png
my_video/frame1100.png
my_video/frame1101.png
my_video/frame1102.png
my_video/frame1103.png
my_video/frame1104.png
my_video/frame1105.png
my_video/frame1106.png
my_video/frame1107.png
my_video/frame1108.png
my_video/frame1109.png
my_video/frame1110.png
my_video/frame1111.png
my_video/frame1112.png
my_video/fr